In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import helper, op, script, tx

### Exercise 1




#### Make [this test](/edit/session5/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [6]:
# Exercise 1

reload(tx)
run(tx.TxTest('test_verify_p2pkh'))

.
----------------------------------------------------------------------
Ran 1 test in 0.490s

OK


In [3]:
# Transaction Construction Example
from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import Tx, TxIn, TxOut
# Step 1
tx_ins = []
prev_tx = bytes.fromhex('8be2f69037de71e3bc856a6627ed3e222a7a2d0ce81daeeb54a3aea8db274149')
prev_index = 4
tx_ins.append(TxIn(prev_tx, prev_index))
# Step 2
tx_outs = []
h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
tx_outs.append(TxOut(
    amount=int(0.38*100000000),
    script_pubkey=p2pkh_script(h160),
))
h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
tx_outs.append(TxOut(
    amount=int(0.1*100000000),
    script_pubkey=p2pkh_script(h160),
))
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)
# Step 3
z = tx_obj.sig_hash(0)
pk = PrivateKey(secret=8675309)
der = pk.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = pk.point.sec()
tx_obj.tx_ins[0].script_sig = Script([sig, sec])
print(tx_obj.serialize().hex())

0100000001494127dba8aea354ebae1de80c2d7a2a223eed27666a85bce371de3790f6e28b040000006b483045022100fa3032607b50e8cb05bedc9d43f986f19dedc22e61320b9765061c5cd9c66946022072d514ef637988515bfa59a660596206de68f0ed4090d0a398e70f4d81370dfb012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67ffffffff0280d54302000000001976a914d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f88ac80969800000000001976a914507b27411ccf7f16f10297de6cef3f291623eddf88ac00000000


### Exercise 2




#### Make [this test](/edit/session5/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [13]:
# Exercise 2

reload(tx)
run(tx.TxTest('test_sign_input'))

.
----------------------------------------------------------------------
Ran 1 test in 0.279s

OK


### Exercise 3
Send 0.04 TBTC to this address

`mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv`

#### Go here to send your transaction: https://blockstream.info/testnet/tx/push


In [18]:
# Exercise 3

from tx import Tx, TxIn, TxOut
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script
prev_tx = bytes.fromhex('8f5a634245870352c88dbd39073ccc8024cb2a155d6398b1d8e0542f616e657c')  # CHANGE
prev_index = 8  # CHANGE
target_address = 'mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv'
target_amount = 0.04
fee = 50000
passphrase = b'sashafklein@gmail.com:funnypassphrase'  # CHANGE
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
change_address = private_key.point.address(testnet=True)

# initialize inputs
tx_ins = []

# create a new tx input with prev_tx, prev_index
tx_in = TxIn(prev_tx, prev_index)
tx_ins.append(tx_in)

# initialize outputs
tx_outs = []

# decode the hash160 from the target address
target_hash160 = decode_base58(target_address)

# convert hash160 to p2pkh script
target_p2pkh = p2pkh_script(target_hash160)

# convert target amount to satoshis (multiply by 100 million)
target_amount = int(target_amount * 100000000)

# create a new tx output for target with amount and script_pubkey
target_out = TxOut(
    amount=target_amount,
    script_pubkey=target_p2pkh,
)

# decode the hash160 from the change address
change_hash160 = decode_base58(change_address)

# convert hash160 to p2pkh script
change_p2pkh = p2pkh_script(change_hash160)

# get the value for the transaction input (remember testnet=True)
value = tx_in.value(testnet=True)

# calculate change_satoshis based on previous amount, target_satoshis & fee
change = value - target_amount - fee

# create a new tx output for target with amount and script_pubkey
change_out = TxOut(
    amount=change,
    script_pubkey=change_p2pkh,
)

tx_outs.append(target_out)
tx_outs.append(change_out)

# create the transaction (name it tx_obj and set testnet=True)
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)

# now sign the 0th input with the private_key using sign_input
tx_obj.sign_input(0, private_key)

# SANITY CHECK: change address corresponds to private key
if private_key.point.address(testnet=True) != change_address:
    raise RuntimeError('Private Key does not correspond to Change Address, check priv_key and change_address')
# SANITY CHECK: output's script_pubkey is the same one as your address
if tx_ins[0].script_pubkey(testnet=True).commands[2] != decode_base58(change_address):
    raise RuntimeError('Output is not something you can spend with this private key. Check that the prev_tx and prev_index are correct')
# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError(f'Check that the change amount is reasonable. Fee is {tx_obj.fee()}')

serialized = tx_obj.serialize()
hexed = serialized.hex()
print(hexed)
# serialize and hex()


01000000017c656e612f54e0d8b198635d152acb2480cc3c0739bd8dc85203874542635a8f080000006a47304402206646357534e6529697b6e3ff774e10530761ca51948f20af188184714dbfeb5902205b5efe0abc2d69882b71afdec793edcafbff842c921b97361c094642771363a501210396b9578d8782e7f29c0562e20234ed5bf3ba47b4a98af9baa1c820249715eedaffffffff0200093d00000000001976a9146e13971913b9aa89659a9f53d327baa8826f2d7588ac30ca5a00000000001976a914e968eda63cead5463b83804dacb3f06ba4120f8a88ac00000000


### Exercise 4
#### Bonus Question. Only attempt if you've finished Exercise 3 and have time to try it.

Get some testnet coins and spend both outputs (one from your change address and one from the testnet faucet) to

`mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv`

#### You can get some free testnet coins at: https://testnet.coinfaucet.eu/en/


In [28]:
# Exercise 4

# Bonus
from tx import Tx, TxIn, TxOut
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script
prev_tx_1 = bytes.fromhex('99d3093589794a886f4007303ad3d28e5990604f84cd94f30403100c6c913727')  # CHANGE
prev_index_1 = 1  # CHANGE
prev_tx_2 = bytes.fromhex('8a1a62269f05aa82895f48191a0d6b6cebe307fc8a83b9a893319c0dac02e4b9')  # CHANGE
prev_index_2 = 1  # CHANGE
target_btc = 0.02
faucet_btc = 0.01
target_address = 'mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv'
faucet_address = 'mkHS9ne12qx9pS9VojpwU5xtRd4T7X7ZUt'
fee = 50000
passphrase = b'sashafklein@gmail.com:funnypassphrase'  # CHANGE
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
change_address = private_key.point.address(testnet=True)

# initialize inputs
tx_ins = []

# create a new tx input with prev_tx, prev_index
tx_in_1 = TxIn(prev_tx_1, prev_index_1)
tx_in_2 = TxIn(prev_tx_2, prev_index_2)
tx_ins.append(tx_in_1)
tx_ins.append(tx_in_2)

# initialize outputs
tx_outs = []

# decode the hash160 from the target address
target_hash160 = decode_base58(target_address)

# convert hash160 to p2pkh script
target_p2pkh = p2pkh_script(target_hash160)

# convert target amount to satoshis (multiply by 100 million)
target_amount = int(target_btc * 100000000)

# create a new tx output for target with amount and script_pubkey
target_out = TxOut(
    amount=target_amount,
    script_pubkey=target_p2pkh,
)

# decode the hash160 from the change address
faucet_hash160 = decode_base58(faucet_address)

# convert hash160 to p2pkh script
faucet_p2pkh = p2pkh_script(faucet_hash160)

faucet_amount = int(faucet_btc * 100000000)

faucet_out = TxOut(
    amount=faucet_amount,
    script_pubkey = faucet_p2pkh
)

# decode the hash160 from the change address
change_hash160 = decode_base58(change_address)

# convert hash160 to p2pkh script
change_p2pkh = p2pkh_script(change_hash160)

# get the value for the transaction input (remember testnet=True)
value = tx_in_1.value(testnet=True) + tx_in_2.value(testnet=True)

# calculate change_satoshis based on previous amount, target_satoshis & fee
change = value - target_amount - faucet_amount - fee
print(target_amount)
print(faucet_amount)
print(fee)
print(change)
print(value)

# create a new tx output for target with amount and script_pubkey
change_out = TxOut(
    amount=change,
    script_pubkey=change_p2pkh,
)

tx_outs.append(target_out)
tx_outs.append(faucet_out)
tx_outs.append(change_out)

# create the transaction (name it tx_obj and set testnet=True)
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)

# now sign the 0th input with the private_key using sign_input
tx_obj.sign_input(0, private_key)
tx_obj.sign_input(1, private_key)

# SANITY CHECK: change address corresponds to private key
if private_key.point.address(testnet=True) != change_address:
    raise RuntimeError('Private Key does not correspond to Change Address, check priv_key and change_address')
# SANITY CHECK: output's script_pubkey is the same one as your address
if tx_ins[0].script_pubkey(testnet=True).commands[2] != decode_base58(change_address):
    raise RuntimeError('Output is not something you can spend with this private key. Check that the prev_tx and prev_index are correct')
# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError(f'Check that the change amount is reasonable. Fee is {tx_obj.fee()}')

serialized = tx_obj.serialize()
hexed = serialized.hex()
print(hexed)
# serialize and hex()

2000000
1000000
50000
3000000
6050000
01000000022737916c0c100304f394cd844f6090598ed2d33a3007406f884a79893509d399010000006a473044022019637cfc23b80cccf2d728ef4a239a6daaf8e86534acc60848a34e6f3733810102207f48e2c57231fbf32a249d184d57cdca4828be14a20698c74096f1dc97f50ae401210396b9578d8782e7f29c0562e20234ed5bf3ba47b4a98af9baa1c820249715eedaffffffffb9e402ac0d9c3193a8b9838afc07e3eb6c6b0d1a19485f8982aa059f26621a8a010000006b48304502210088909afc62b7dd097421f37a4360b5dad41185097f0f69a8496c442f142af18802200947253599b7e0086cced4979ee7682814845ea14130d13ee5500a6dcf76b64d01210396b9578d8782e7f29c0562e20234ed5bf3ba47b4a98af9baa1c820249715eedaffffffff0380841e00000000001976a9146e13971913b9aa89659a9f53d327baa8826f2d7588ac40420f00000000001976a914344a0f48ca150ec2b903817660b9b68b13a6702688acc0c62d00000000001976a914e968eda63cead5463b83804dacb3f06ba4120f8a88ac00000000


In [ ]:
# op_checkmultisig
def op_checkmultisig(stack, z):
    if len(stack) < 1:
        return False
    n = decode_num(stack.pop())
    if len(stack) < n + 1:
        return False
    sec_pubkeys = []
    for _ in range(n):
        sec_pubkeys.append(stack.pop())
    m = decode_num(stack.pop())
    if len(stack) < m + 1:
        return False
    der_signatures = []
    for _ in range(m):
        # signature is assumed to be using SIGHASH_ALL
        der_signatures.append(stack.pop()[:-1])
    # OP_CHECKMULTISIG bug
    stack.pop()
    try:
        raise NotImplementedError
    except (ValueError, SyntaxError):
        return False
    return True

### Exercise 5




#### Make [this test](/edit/session5/op.py) pass: `op.py:OpTest:test_op_checkmultisig`

In [33]:
# Exercise 5

reload(op)
reload(script)
run(op.OpTest('test_op_checkmultisig'))

S256Point(b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb71,ea9a3d686077906ff4ed2c0d1a7e29d4d04b72b4764ae40c89992181f47e1429)


.
----------------------------------------------------------------------
Ran 1 test in 0.376s

OK


### Exercise 6
Find the hash160 of the RedeemScript
```
5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae
```


In [36]:
# Exercise 6

from helper import hash160
hex_redeem_script = '5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae'
# bytes.fromhex script
bytes_redeem_script = bytes.fromhex(hex_redeem_script)

# hash160 result
hash_160 = hash160(bytes_redeem_script)

# hex() to display
print(hash_160.hex())


74d691da1574e6b3c192ecfb52cc8984ee7b6c56


In [37]:
# P2SH address construction example
from helper import encode_base58_checksum
print(encode_base58_checksum(b'\x05'+bytes.fromhex('74d691da1574e6b3c192ecfb52cc8984ee7b6c56')))

3CLoMMyuoDQTPRD3XYZtCvgvkadrAdvdXh


### Exercise 7




#### Make [this test](/edit/session5/helper.py) pass: `helper.py:HelperTest:test_p2pkh_address`

In [38]:
# Exercise 7

reload(helper)
run(helper.HelperTest('test_p2pkh_address'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 8




#### Make [this test](/edit/session5/helper.py) pass: `helper.py:HelperTest:test_p2sh_address`

In [39]:
# Exercise 8

reload(helper)
run(helper.HelperTest('test_p2sh_address'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 9




#### Make [this test](/edit/session5/script.py) pass: `script.py:ScriptTest:test_address`

In [41]:
# Exercise 9

reload(script)
run(script.ScriptTest('test_address'))

E
ERROR: test_address (script.ScriptTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/sasha/code/btc/pb-exercises/session5/script.py", line 221, in test_address
    self.assertEqual(p2pkh_script_pubkey.address(), address_1)
  File "/Users/sasha/code/btc/pb-exercises/session5/script.py", line 198, in address
    raise NotImplementedError
NotImplementedError

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


In [42]:
# z for p2sh example
from helper import hash256
h256 = hash256(bytes.fromhex('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(h256, 'big')
print(hex(z))

0xe71bfa115715d6fd33796948126f40a8cdd39f187e4afb03896795189fe1423c


In [43]:
# p2sh verification example
from ecc import S256Point, Signature
from helper import hash256
h256 = hash256(bytes.fromhex('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(h256, 'big')
point = S256Point.parse(bytes.fromhex('022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb70'))
sig = Signature.parse(bytes.fromhex('3045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a89937'))
print(point.verify(z, sig))

True


### Exercise 10
Validate the second signature of the first input

```
0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000db00483045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a8993701483045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c568700000000
```

The sec pubkey of the second signature is:
```
03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb71
```

The der signature of the second signature is:
```
3045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022
```

The redeemScript is:
```
475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae
```


In [47]:
# Exercise 10

from io import BytesIO
from ecc import S256Point, Signature
from helper import int_to_little_endian, SIGHASH_ALL
from script import Script
from tx import Tx
hex_sec = '03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb71'
hex_der = '3045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e754022'
hex_redeem_script = '475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae'
sec = bytes.fromhex(hex_sec)
der = bytes.fromhex(hex_der)
redeem_script_stream = BytesIO(bytes.fromhex(hex_redeem_script))
redeem_script = Script.parse(redeem_script_stream)
hex_tx = '0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000db00483045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a8993701483045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c568700000000'
stream = BytesIO(bytes.fromhex(hex_tx))

# parse the S256Point and Signature
point = S256Point.parse(sec)
sig = Signature.parse(der)

# parse the Tx
tx_obj = Tx.parse(stream)

# change the first input's ScriptSig to RedeemScript
tx_obj.tx_ins[0].script_sig = redeem_script

# get the serialization
serialized = tx_obj.serialize()

# add the sighash (4 bytes, little-endian of SIGHASH_ALL)
serialized += int_to_little_endian(SIGHASH_ALL, 4)

# hash256 the result

# your z is the hash256 as a big-endian number: use int.from_bytes(x, 'big')
z = int.from_bytes(hash256(serialized), 'big')

# now verify the signature using point.verify
point.verify(z, sig)

True